In [2]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig, TextClassificationPipeline
from transformers import pipeline
import numpy as np
import pandas as pd
from scipy.special import softmax
import datetime

In [3]:
!pip install torch

In [4]:
!pip install protobuf==3.20.0

In [5]:
data1 = pd.read_csv('complete_vol_2023-03-14.csv')
#data = data[['text', 'datetime']]

In [6]:
data1.head()
data1 = data1.drop(['Unnamed: 0', 'daily_tweets', 'MA7_Sentiment', 'MA20_Sentiment', 'MA50_Sentiment', 'BTC_price', 'BTC_trading_volume'], axis =1)

In [7]:
data1

,date,negative_bert,neutral_bert,positive_bert
0,2021-01-01,0.085275,0.489181,0.425545
1,2021-01-02,0.143365,0.439075,0.417560
2,2021-01-03,0.148220,0.432023,0.419757
3,2021-01-04,0.101558,0.507526,0.390916
4,2021-01-05,0.088530,0.517456,0.394014
...,...,...,...,...
799,2023-03-11,0.118417,0.483141,0.398441
800,2023-03-12,0.137051,0.479557,0.383392
801,2023-03-13,0.126823,0.496415,0.376762
802,2023-03-14,0.101771,0.503114,0.395115


In [20]:
#applying most recent data
data = pd.read_csv('16-03-23_to_08-04-23.csv')

In [21]:
def preprocess(text):
    new_text = []
    text = str(text)
    text = text.replace("\n", " ")
    for t in text.split(" "):
        t = '' if t.startswith('@') and len(t) > 1 else t
        t = '' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [22]:
data['process_text'] = data.text
# data['negative_bert'] = data.text
# data['neutral_bert'] = data.text
# data['positive_bert'] = data.text

#use the preprocess to clean the data
data['process_text'] = data['text'].apply(preprocess)

In [46]:
data

,date,process_text,negative_bert,neutral_bert,positive_bert
0,2023-03-16,＃BTC ＃Bitcoin,0.489502,0.275285,0.235213
1,2023-03-16,Current #Bitcoin Price is $24399 #BTC #Crypto,0.006015,0.549235,0.444750
2,2023-03-16,Elon is blinded by his ego when it comes to ...,0.493548,0.283148,0.223303
3,2023-03-16,Excellent work congratulations #BITCOIN #BTC,0.003711,0.473139,0.523150
4,2023-03-16,#Bitcoin Fear and Greed index Current #BTC pr...,0.004060,0.978413,0.017527
...,...,...,...,...,...
10115,2023-04-08,"Bitcoin - BTC Price: $28,028.82 Change in 1h: ...",0.010006,0.800126,0.189869
10116,2023-04-08,One Bitcoin now worth $28036.125. Market Cap $...,0.000271,0.855259,0.144470
10117,2023-04-08,"#Bitcoin is currently $28,036.1254",0.068032,0.431275,0.500692
10118,2023-04-08,One Bitcoin now worth $28048.00@bitstamp. High...,0.000704,0.864073,0.135222


In [24]:
!pyenv local crypto

In [25]:
#Model 2b - Use the bert model, with full scores
MODEL_bert = f"ElKulako/cryptobert"
tokenizer_bert = AutoTokenizer.from_pretrained(MODEL_bert)
tokenizer_bert.model_max_length = 512 #solve the error: RuntimeError: The expanded size of the tensor (562) must match the existing size (514) at non-singleton dimension
config_bert = AutoConfig.from_pretrained(MODEL_bert)


# PT
model_bert = AutoModelForSequenceClassification.from_pretrained(MODEL_bert)
model_bert.config.max_position_embeddings = 512

In [26]:
def scores_bert(sample_text):
    encoded_input_bert = tokenizer_bert(sample_text, return_tensors='pt')
    output_bert = model_bert(**encoded_input_bert)
    scores_bert = output_bert[0][0].detach().numpy()
    scores_bert = softmax(scores_bert) #1st score is negative, 2nd score is netural, 3rd score is positive
    return scores_bert

In [27]:
data['text'] = data['process_text'].apply(scores_bert)

In [51]:
data[10:20]

,date,process_text,negative_bert,neutral_bert,positive_bert
10,2023-03-16,"#Bitcoin 🙌Season defined as scores below 25, ...",0.024030,0.834113,0.141857
11,2023-03-16,"Fidelity #Crypto quietly went live, giving mil...",0.000070,0.577342,0.422588
12,2023-03-16,[NEW] New Moons Distribution (Round 37 Proposa...,0.000190,0.251336,0.748474
13,2023-03-16,Don't miss your chance to stay ahead in #crypt...,0.000087,0.890030,0.109883
14,2023-03-16,Thoughts on what the Fed has as current option...,0.000927,0.471062,0.528011
15,2023-03-16,Starting an almost daily Twitter Spaces very s...,0.000261,0.241797,0.757943
16,2023-03-16,We just posted our first ever BitTalk intervie...,0.000121,0.836693,0.163186
17,2023-03-16,#Bitcoin is safer than banks also .,0.052349,0.508737,0.438915
18,2023-03-16,#Bitcoin 2d closed with the biggest sell volum...,0.398228,0.566411,0.035360
19,2023-03-16,🔥 ✅TS ❕ ATOMUSD: descending channel✅❓ Check th...,0.000262,0.679075,0.320663


In [52]:
for i in range(len(data['process_text'])):
    print(f"{i}-------------------")
    print(data['process_text'][i])
    if i > 20:
        break

0-------------------
＃BTC ＃Bitcoin 
1-------------------
Current #Bitcoin Price is $24399 #BTC #Crypto
2-------------------
  Elon is blinded by his ego when it comes to #Bitcoin. He got it right the first time when he backed Bitcoin, then decided he wanted his own crypto which is #Doge-shit.
3-------------------
 Excellent work congratulations #BITCOIN #BTC
4-------------------
#Bitcoin Fear and Greed index  Current #BTC price is 24288 $  #Crypto #FearAndGreed  (6/13) 474180 
5-------------------
Here’s how the metaverse will evolve 🫣 #metaverse #cryptoworld #Meta #cryptocurrency #blockchain #crypto #nft #web3 #ethereum #ETH #bitcoin #BTC #nftcommunity #nftcollector 
6-------------------
10/ For similar threads on #Bitcoin #crypto  #DeFi #NFT #web3 :  Follow 👉   #Bitcoin #crypto #cryptocurrency #nfts #web3 #web3gaming #web3community #NFTCommunity #ETH
7-------------------
Ethereum Is One Step Away From Staked ETH Withdrawals  #Ethereum #News #crypto #Bitcoin #Coins
8------------------

In [42]:
data = data[['datetime','process_text','text']]
data

KeyError: "['datetime', 'text'] not in index"

In [41]:
#modifying the datetime
def dateonly(x):
    return x[:10]
data['date'] = data['datetime'].apply(dateonly)

data

KeyError: 'datetime'

In [43]:
data["negative_bert"] = data["text"].map(lambda x: x[0])
data["neutral_bert"] = data["text"].map(lambda x: x[1])
data["positive_bert"] = data["text"].map(lambda x: x[2])
data = data[['date', 'process_text', 'negative_bert', 'neutral_bert','positive_bert']]
data

KeyError: 'text'

In [44]:
grouped_data = pd.DataFrame(data.groupby(['date'])[['negative_bert', 'neutral_bert', 'positive_bert']].sum().fillna(0).reset_index())
grouped_data = grouped_data.rename_axis("", axis="columns")

In [45]:
data1

,date,negative_bert,neutral_bert,positive_bert
0,2021-01-01,0.085275,0.489181,0.425545
1,2021-01-02,0.143365,0.439075,0.417560
2,2021-01-03,0.148220,0.432023,0.419757
3,2021-01-04,0.101558,0.507526,0.390916
4,2021-01-05,0.088530,0.517456,0.394014
...,...,...,...,...
799,2023-03-11,0.118417,0.483141,0.398441
800,2023-03-12,0.137051,0.479557,0.383392
801,2023-03-13,0.126823,0.496415,0.376762
802,2023-03-14,0.101771,0.503114,0.395115


In [35]:
concatenated_df = pd.concat([data1, grouped_data], ignore_index=True)

# Write the concatenated DataFrame to a new CSV file
concatenated_df.to_csv('concatenated_file.csv', index=False)

data_concat = pd.read_csv('concatenated_file.csv')

In [36]:
def data_prep(df):
    
    X = df[['date']]
    X['date'] = pd.to_datetime(X['date'])
    
    X['Daily_Average_Sentiment']=df['positive_bert']/df.iloc[:,1:].sum(axis=1)
    
    
    X['MA20_Sentiment'] = X['Daily_Average_Sentiment'].rolling(window=21).mean()
    X['MA50_Sentiment'] = X['Daily_Average_Sentiment'].rolling(window=51).mean()
    
    X['Threshold_50'] = X['Daily_Average_Sentiment']>X['MA50_Sentiment']
    X['Threshold_20'] = X['Daily_Average_Sentiment']>X['MA20_Sentiment']
    X['Buy_Sell'] = X['Threshold_50'] & X['Threshold_20']
    
    NNX= X[['date','Daily_Average_Sentiment', 'MA50_Sentiment','MA20_Sentiment','Threshold_50','Threshold_20',
             'Buy_Sell']]
    
    return NNX

In [37]:
final_data = data_prep(data_concat)

/var/folders/dn/rwn91pwx3q9c4zbfkf6rfb6h0000gn/T/ipykernel_46326/815910447.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['date'] = pd.to_datetime(X['date'])
/var/folders/dn/rwn91pwx3q9c4zbfkf6rfb6h0000gn/T/ipykernel_46326/815910447.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Daily_Average_Sentiment']=df['positive_bert']/df.iloc[:,1:].sum(axis=1)
/var/folders/dn/rwn91pwx3q9c4zbfkf6rfb6h0000gn/T/ipykernel_46326/815910447.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [38]:
final_data[-50:]

,date,Daily_Average_Sentiment,MA50_Sentiment,MA20_Sentiment,Threshold_50,Threshold_20,Buy_Sell
778,2023-02-18,0.360798,0.362003,0.364323,False,False,False
779,2023-02-19,0.371081,0.362217,0.364365,True,True,True
780,2023-02-20,0.352773,0.362252,0.364158,False,False,False
781,2023-02-21,0.344455,0.362216,0.364191,False,False,False
782,2023-02-22,0.341220,0.362054,0.363353,False,False,False
783,2023-02-23,0.361243,0.361679,0.363494,False,False,False
784,2023-02-24,0.345896,0.361514,0.362666,False,False,False
785,2023-02-25,0.331820,0.361718,0.360262,False,False,False
786,2023-02-26,0.371403,0.362005,0.361744,True,True,True
787,2023-02-27,0.345592,0.361837,0.361309,False,False,False
